In [1]:
import pandas as pd
import numpy as np

In [2]:
temp_df_pt1 = pd.read_csv('../actual_temperature_data/gridded_averages_1880-2021/NOAAGlobalTemp_v5a_1.csv')
temp_df_pt2 = pd.read_csv('../actual_temperature_data/gridded_averages_1880-2021/NOAAGlobalTemp_v5a_2.csv')

# concatenate dataframes
# gonna turn all the wrong years into -1
frames = [temp_df_pt1, temp_df_pt2]
temp_df = pd.concat(frames, sort=False)

tree_df = pd.read_csv('../tree_data.csv')
tree_df['year'] = tree_df['year'].apply(lambda x: float(x) if str(x).isdigit() else -1)
tree_df.head()

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,avg_tree_ring_width,lat,long,elevation,tree_species,organism_group
0,969.0,164.5,51.75041,-0.34223,NaN,Quercus spp. L.,0
1,970.0,188.5,51.75041,-0.34223,NaN,Quercus spp. L.,0
2,971.0,139.0,51.75041,-0.34223,NaN,Quercus spp. L.,0
3,972.0,173.0,51.75041,-0.34223,NaN,Quercus spp. L.,0
4,973.0,221.0,51.75041,-0.34223,NaN,Quercus spp. L.,0


In [26]:
tree_df['tree_species'].value_counts()

Pseudotsuga menziesii (Mirb.) Franco                                            214694
Pinus sylvestris L.                                                             152902
Tsuga mertensiana (Bong.) Carriere                                              122337
Pinus ponderosa Douglas ex C. Lawson                                            112764
Picea glauca (Moench) Voss                                                       85047
Pinus edulis Engelm.                                                             62735
Picea abies (L.) H. Karst.                                                       50093
Pinus flexilis E. James                                                          45869
Pinus longaeva D.K. Bailey = Pinus aristata var. longaeva                        45638
Larix sibirica Ledeb. = Larix russica (Endl.) Sabine ex Trautv.                  40882
Sequoiadendron giganteum (Lindl.) Buchholz                                       39378
Quercus spp. L.                            

In [3]:
# How many have bad years
print(tree_df.shape)
tree_df[tree_df['year'] == -1].shape

(2062155, 7)


(120637, 7)

In [4]:
def get_5x5_grid(og_lat, og_lon, base=2.5):
    lat1 = base * round((og_lat-2.5)/base)
    lat2 = base * round((og_lat+2.5)/base)

    lon1 = base * round((og_lon-2.5)/base)
    lon2 = base * round((og_lon+2.5)/base)

    if lat2-2.5 < og_lat:
        lat2 += 2.5
        lat1 += 2.5
    else:
        lat2 -= 2.5
        lat1 -= 2.5

    if lon2-2.5 < og_lon:
        lon2 += 2.5
        lon1 += 2.5
    else:
        lon2 -= 2.5
        lon1 -= 2.5

    if (lat1 % 5 )== 0 or (lat2 % 5 )== 0:
        if lat1 < 0:
            lat1 += 2.5
            lat2 += 2.5
        else: 
            lat1 -= 2.5
            lat2 -= 2.5

    if (lon1 % 5 )== 0 or (lon2 % 5 )== 0:
        lon1 += 2.5
        lon2 += 2.5

    if (lat1 == -92.5 and lat2 == -87.5):
        lat1 = -87.5
        lat2 = -82.5
    elif (lat1 == 87.5 and lat2 == 92.5):
        lat1 = 82.5
        lat2 = 87.5
    
    if (lon1 == 0.0 and lon2 == 5.0):
        lon1 = 2.5
        lon2 = 7.5
    elif (lon1 == 355.0 and lon2 == 360.0):
        lon1 = 352.5
        lon2 = 357.5

    return lat1, lat2, lon1, lon2

In [5]:
temp_df['year'] = temp_df['time'].apply(lambda x: str(x)[0:4])
temp_df = temp_df.dropna(subset=['anom'])
temp_annual_df =  temp_df.groupby(['year', 'lat', 'lon']).agg({'anom': ['mean']}).reset_index()

In [31]:
# given an observation row, return the row with the correct 5x5 temperature anomaly

def get_gridded_temp_anom(lat, lon, year):
    lat1, lat2, lon1, lon2 = get_5x5_grid(lat, lon)
    temp_df_for_lat_lon = temp_df[((temp_df['lat'] == lat1) | (temp_df['lat'] == lat2)) &
                                  ((temp_df['lon'] == lon1) | (temp_df['lon'] == lon2)) &
                                  (temp_df['year'] == str(year))]
    annual_temp_df_for_lat_lon = temp_df_for_lat_lon.groupby(['year']).agg({'anom': ['mean']}).reset_index()
    if len(annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()) > 0:
        temp_anom = annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()[0]
        return temp_anom

In [33]:
# run it on the data

tree_df_temp = tree_df[tree_df['year'] > 1879]
tree_df_temp = tree_df_temp[tree_df_temp['year'] < 2021]
tree_df_temp = tree_df_temp[tree_df_temp['tree_species'] == 'Abies alba Mill.']
tree_df_temp['temp_anom'] = tree_df.apply(lambda x: get_gridded_temp_anom(x['lat'], x['long'], x['year']), axis=1)
tree_df_temp